# Spaceship titanic  
캐글의 유명한 예제인 타이타닉과 거의 동일하다고 할 수 있는 예제라고 할 수 있다.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train=pd.read_csv("data/spaceship-titanic/train.csv")
test=pd.read_csv("data/spaceship-titanic/test.csv")

In [3]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


**각각의 특성에 대해서 알아보자.**
1. Passengerld: 승객의 ID
2. HomePlanet: 출신 행성
3. CryoSleep: 냉동 수면
4. Cabin: 객실
5. destination: 목적지
6. Transported: 차원이동여부
7. VRDeck: Deck은 갑판을 의미하는데 VRDeck이 정확히 무엇을 의미하는지는 모르겠다. 

## 데이터 알아보기

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [6]:
train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [7]:
train["HomePlanet"].value_counts()

Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64

전체 8693명 중 출신 행성이 지구인 사람은 4602명, 유로파위성인 사람은 2131명, 화성인 사람은 1759명이고 출신 행성 데이터가 없는 사람은 201명임을 알 수 있다.  
<br/>
우주선에서 소비한 금액을 하나로 합친다.

In [8]:
train["Expense"]=train["RoomService"]+train["FoodCourt"]+train["ShoppingMall"]+train["Spa"]+train["VRDeck"]
test["Expense"]=test["RoomService"]+test["FoodCourt"]+test["ShoppingMall"]+test["Spa"]+test["VRDeck"]

### 데이터 선별하기  
제출용 passangerID 미리제외

In [9]:
ID=test['PassengerId']

**'CryoSleep','VIP,'Transported','Expense'**  
*유의미한 데이터로 추정*

처음 데이터를 위와 같이 선별했는데 Age도 Transported 유의미한 상관관계가 있을까하여 한번 분석해보려고한다.

범주형데이터끼리의 상관관계를 나타내기위해 age를 구간별로 나누었다.

In [10]:
train['Age']=pd.cut(train['Age'],bins=[0, 15, 25, 35, 60, 100],labels=['어린이', '청년', '장년', '중년', '노년'])

In [11]:
from scipy.stats import chi2_contingency

pd.crosstab(train['Age'],train['Transported'])

Transported,False,True
Age,,
어린이,402,660
청년,1410,1246
장년,1088,1017
중년,1176,1117
노년,116,104


딱히 유의미해보이지 않아서 제외...

In [12]:
train=train[["CryoSleep","VIP","Transported","Expense"]]
test=test[["CryoSleep","VIP","Expense"]]

In [13]:
train.head()

,CryoSleep,VIP,Transported,Expense
0,False,False,False,0.0
1,False,False,True,736.0
2,False,True,False,10383.0
3,False,False,False,5176.0
4,False,False,True,1091.0


편의상 결측값은 전부 False로 판단

In [14]:
train=train.replace(False,0)
train=train.replace(True,1)
train=train.fillna(0)

test=test.replace(False,0)
test=test.replace(True,1)
test=test.fillna(0)

In [15]:
x_train=train.drop("Transported",axis=1)
y_train=train['Transported']

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

#로지스틱회귀
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)
print(f"LogisticRegression: {log_reg.score(x_train, y_train)}")

#서포트벡터머신
svc = SVC()
svc.fit(x_train, y_train)
print(f"SupportVectorMachine: {svc.score(x_train, y_train)}")

#랜덤포레스트
random_forest=RandomForestClassifier(n_estimators=30)
random_forest.fit(x_train, y_train)
print(f"RandomForest: {random_forest.score(x_train, y_train)}")

#K최근접이웃
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train, y_train)
print(f"KNN: {knn.score(x_train, y_train)}")

LogisticRegression: 0.718279075117911
SupportVectorMachine: 0.7113769699758427
RandomForest: 0.7930518808236512
KNN: 0.7519843552283446


In [ ]:
random_forest=RandomForestClassifier(n_estimators=30)
random_forest.fit(x_train, y_train)
y_pred=random_forest.predict(test)
Y_pred=[]
for i in y_pred:
    if i==1.0:
        Y_pred.append(True)
    else:
        Y_pred.append(False)

result=pd.DataFrame({
    "PassengerId": ID,
    "Transported": Y_pred
})
result.to_csv('result.csv',index=False)

제출 점수는 Score: 0.68903이다.  
(차후 개선할 예정)